In [2]:
! pip install awswrangler


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import awswrangler as wr
import pandas as pd

In [4]:
df = wr.s3.read_parquet("s3://<INSERT_YOUR_BUCKET_NAME>/raw/year=2023/month=08/day=07/hour=13/retailer_id=retailer-0/Demo-StockUpdates-10-2023-08-07-13-30-24-7663efb9-d5b4-3a15-abd9-abd452a5b253.parquet")

In [5]:
df

,accountid,retailerid,channelid,productid,articleid,articlenumber,lastupdated,quantity
0,10,retailer-0,10,850,4,666,'2023-08-07T13:30:23.920+00:00',78
1,1,retailer-0,10,444,1,849,'2023-08-07T13:30:23.914+00:00',56
2,10,retailer-0,7,409,1,953,'2023-08-07T13:30:23.917+00:00',12
3,1,retailer-0,10,520,1,319,'2023-08-07T13:30:23.918+00:00',62
4,1,retailer-0,3,518,2,960,'2023-08-07T13:30:23.919+00:00',100
...,...,...,...,...,...,...,...,...
51564,8,retailer-0,2,8,0,523,'2023-08-07T13:36:47.568+00:00',31
51565,9,retailer-0,3,91,1,94,'2023-08-07T13:36:47.572+00:00',36
51566,1,retailer-0,3,49,0,35,'2023-08-07T13:36:47.556+00:00',5
51567,5,retailer-0,3,111,3,9,'2023-08-07T13:36:47.566+00:00',0


In [6]:
df.groupby(['accountid', 'articleid']).size()

accountid  articleid
0          0            953
           1            922
           2            960
           3            944
           4            938
1          0            945
           1            944
           2            925
           3            968
           4            911
2          0            884
           1            977
           2            986
           3            925
           4            954
3          0            930
           1            939
           2            936
           3            938
           4            989
4          0            959
           1            952
           2            928
           3            975
           4            919
5          0            872
           1            928
           2            988
           3            904
           4            963
6          0            912
           1            964
           2            944
           3            960
           4            949

In [7]:
df[(df['accountid'] == 0) & (df['articleid'].isin([1, 2]))].sort_values(by='lastupdated').tail(5)

,accountid,retailerid,channelid,productid,articleid,articlenumber,lastupdated,quantity
51470,0,retailer-0,6,826,2,150,'2023-08-07T13:36:47.558+00:00',46
51428,0,retailer-0,10,577,1,897,'2023-08-07T13:36:47.560+00:00',38
51545,0,retailer-0,0,619,1,849,'2023-08-07T13:36:47.563+00:00',5
51505,0,retailer-0,3,614,1,750,'2023-08-07T13:36:47.566+00:00',12
51507,0,retailer-0,3,267,2,431,'2023-08-07T13:36:47.566+00:00',48


In [8]:
# Sort the data frame by accountid/brand id
df_sorted = df.sort_values(['accountid', 'articleid', 'lastupdated'])

# Step 2: Use groupby and aggregation to get the latest stock update per article for each brand
latest_updates = df_sorted.groupby(['accountid', 'articleid'], as_index=False).last()

# Step 3: Organize the data and create the output dictionary
brand_articles = latest_updates.groupby('accountid').apply(lambda x: {
    "account_id": x['accountid'].iloc[0],
    "retailer_id": x['retailerid'].iloc[0],
    "amount_articles": len(x),
    "articles": x[["articleid", "articlenumber", "quantity", "productid", "accountid", "channelid"]].to_dict(orient="records")
}).tolist()

In [9]:
for brand_article in brand_articles:
    print(f"Stock update for brand {brand_article['account_id']} with {brand_article['amount_articles']} entries")
    print(brand_article)

Stock update for brand 0 with 5 entries
{'account_id': 0, 'retailer_id': 'retailer-0', 'amount_articles': 5, 'articles': [{'articleid': 0, 'articlenumber': 222, 'quantity': 69, 'productid': '367', 'accountid': 0, 'channelid': 6}, {'articleid': 1, 'articlenumber': 750, 'quantity': 12, 'productid': '614', 'accountid': 0, 'channelid': 3}, {'articleid': 2, 'articlenumber': 431, 'quantity': 48, 'productid': '267', 'accountid': 0, 'channelid': 3}, {'articleid': 3, 'articlenumber': 75, 'quantity': 76, 'productid': '219', 'accountid': 0, 'channelid': 3}, {'articleid': 4, 'articlenumber': 975, 'quantity': 63, 'productid': '474', 'accountid': 0, 'channelid': 2}]}
Stock update for brand 1 with 5 entries
{'account_id': 1, 'retailer_id': 'retailer-0', 'amount_articles': 5, 'articles': [{'articleid': 0, 'articlenumber': 864, 'quantity': 55, 'productid': '764', 'accountid': 1, 'channelid': 3}, {'articleid': 1, 'articlenumber': 813, 'quantity': 77, 'productid': '516', 'accountid': 1, 'channelid': 0}, 